In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
import time
import mani_skill.env
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style

PATH = 'drawer_data.csv'
ENV_NAME = 'OpenCabinetDrawer-v0'         # 游戏名称
SEED = 123                       # 随机数种子

class PreTrain(object):
    def __init__(self, a_dim, s_dim, a_bound,):
        
        data = pd.read_csv(PATH)
        self.memory = np.array(data)
        
        self.a_dim = a_dim
        self.s_dim = s_dim
        self.a_bound = a_bound
        self.pointer = 0                                                                         # exp buffer指针
        self.lr_a = 0.001                                                                        # learning rate for actor
        self.gamma = 0.9                                                                         # reward discount
        self.tau = 0.01                                                                          # 软更新比例
        self.memory_capacity = 406357
        self.batch_size = 32
#         self.memory = np.zeros((self.memory_capacity, s_dim * 2 + a_dim + 1), dtype=np.float32)

        self.Reward_record = []
        
        class ANet(nn.Module):                               # 定义动作网络
            def __init__(self, s_dim, a_dim, a_bound):
                super(ANet,self).__init__()
                self.a_bound = a_bound
                self.fc1 = nn.Linear(s_dim,512)
                self.fc1.weight.data.normal_(0,0.1)          # initialization
                
                self.fc2 = nn.Linear(512,512)
                self.fc2.weight.data.normal_(0,0.1)
                               
                self.fc3 = nn.Linear(512,512)
                self.fc3.weight.data.normal_(0,0.1)
                
                self.out = nn.Linear(512,a_dim)
                self.out.weight.data.normal_(0,0.1)          # initialization
            def forward(self,x):
                x = self.fc1(x)
                x = F.relu(x)
                
                x = self.fc2(x)
                x = F.relu(x)
                
                x = self.fc3(x)
                x = F.relu(x)
                
                x = self.out(x)
                x = F.tanh(x)
                actions_value = x * a_bound
                return actions_value



        self.Actor_eval = ANet(s_dim, a_dim, a_bound)        # 主网络
        self.atrain = torch.optim.Adam(self.Actor_eval.parameters(),lr = self.lr_a)  # actor的优化器
        self.loss = nn.MSELoss()                          # 损失函数采用均方误差
        
    def train(self):
            
        for i in range(len(self.memory)):
                
            bt = self.memory[i, :]                                              # 采样batch_size个sample
            bs = torch.FloatTensor(bt[:self.s_dim])                                # state
            ba = torch.FloatTensor(bt[self.s_dim: self.s_dim + self.a_dim])        # action
                
            a = self.Actor_eval(bs)
            loss_a = self.loss(ba, a)
            
            self.atrain.zero_grad()
            loss_a.backward()
            self.atrain.step()
        
    def save(self,folder_name):
        os.mkdir('./DPG model/' + folder_name)
        PATH1 = './DPG model/' + folder_name + '/Actor_eval.h5f'
        torch.save(self.Actor_eval.state_dict(), PATH1)
            

In [2]:
###############################  training  ####################################
env = gym.make(ENV_NAME)
env = env.unwrapped
env.seed(SEED)                                          # 设置Gym的随机数种子
torch.manual_seed(SEED)                                 # 设置pytorch的随机数种子

env.set_env_mode(obs_mode='state', reward_type='dense')

s_dim = env.observation_space.shape[0]                  # 状态空间
a_dim = env.action_space.shape[0]                       # 动作空间
a_bound = 1                         # 动作取值区间,对称区间，故只取上界

pre = PreTrain(a_dim, s_dim, a_bound)

t1 = time.time()

pre.train()

pre.save('drawer_actor_pre')
print('Running time: ', time.time() - t1)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/lue/anaconda3/envs/mani_skill/lib/python3.8/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


FileExistsError: [Errno 17] File exists: './DPG model/drawer_actor_pre'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
import time
import mani_skill.env
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style

PATH = 'drawer_data.csv'
ENV_NAME = 'OpenCabinetDrawer-v0'         # 游戏名称
SEED = 123                       # 随机数种子

class PreTrain(object):
    def __init__(self, a_dim, s_dim, a_bound,):
        
        data = pd.read_csv(PATH)
        self.memory = np.array(data)
        
        self.a_dim = a_dim
        self.s_dim = s_dim
        self.a_bound = a_bound
        self.pointer = 0                                                                         # exp buffer指针
        self.lr_c = 0.002                                                                        # learning rate for actor
        self.gamma = 0.9                                                                         # reward discount
        self.tau = 0.01                                                                          # 软更新比例
        self.memory_capacity = 406357
        self.batch_size = 32
#         self.memory = np.zeros((self.memory_capacity, s_dim * 2 + a_dim + 1), dtype=np.float32)

        self.Reward_record = []
    
        class ANet(nn.Module):                               # 定义动作网络
            def __init__(self, s_dim, a_dim, a_bound):
                super(ANet,self).__init__()
                self.a_bound = a_bound
                self.fc1 = nn.Linear(s_dim,256)
                self.fc1.weight.data.normal_(0,0.1)          # initialization
                
                self.fc2 = nn.Linear(256,512)
                self.fc2.weight.data.normal_(0,0.1)
                               
                self.fc3 = nn.Linear(512,256)
                self.fc3.weight.data.normal_(0,0.1)
                
                self.out = nn.Linear(256,a_dim)
                self.out.weight.data.normal_(0,0.1)          # initialization
            def forward(self,x):
                x = self.fc1(x)
                x = F.relu(x)
                
                x = self.fc2(x)
                x = F.relu(x)
                
                x = self.fc3(x)
                x = F.relu(x)
                
                x = self.out(x)
                x = F.tanh(x)
                actions_value = x * a_bound
                return actions_value
    
        class ANet(nn.Module):                               # 定义动作网络
            def __init__(self, s_dim, a_dim, a_bound):
                super(ANet,self).__init__()
                self.a_bound = a_bound
                self.fc1 = nn.Linear(s_dim,256)
                self.fc1.weight.data.normal_(0,0.1)          # initialization
                
                self.fc2 = nn.Linear(256,512)
                self.fc2.weight.data.normal_(0,0.1)
                               
                self.fc3 = nn.Linear(512,256)
                self.fc3.weight.data.normal_(0,0.1)
                
                self.out = nn.Linear(256,a_dim)
                self.out.weight.data.normal_(0,0.1)          # initialization
            def forward(self,x):
                x = self.fc1(x)
                x = F.relu(x)
                
                x = self.fc2(x)
                x = F.relu(x)
                
                x = self.fc3(x)
                x = F.relu(x)
                
                x = self.out(x)
                x = F.tanh(x)
                actions_value = x * a_bound
                return actions_value
        
        class CNet(nn.Module):                               # 定义价值网络
            def __init__(self,s_dim,a_dim):
                super(CNet,self).__init__()
                self.fcs1 = nn.Linear(s_dim,256)
                self.fcs1.weight.data.normal_(0,0.1)          # initialization
                
                self.fcs2 = nn.Linear(256,512)
                self.fcs2.weight.data.normal_(0,0.1) 
                
                self.fcs3 = nn.Linear(512,256)
                self.fcs3.weight.data.normal_(0,0.1) 
                
                self.fca1 = nn.Linear(a_dim,256)
                self.fca1.weight.data.normal_(0,0.1)          # initialization
                
                self.fca2 = nn.Linear(256,512)
                self.fca2.weight.data.normal_(0,0.1)                
                
                self.fca3 = nn.Linear(512,256)
                self.fca3.weight.data.normal_(0,0.1)
                
                self.out = nn.Linear(256,1)
                self.out.weight.data.normal_(0, 0.1)         # initialization
            def forward(self,s,a):
                x = self.fcs1(s)                              # 输入状态
                x = F.relu(x)
                
                x = self.fcs2(x)
                x = F.relu(x)
                
                x = self.fcs3(x)
                x = F.relu(x) 
                
                y = self.fca1(a)                              # 输入动作
                y = F.relu(y)
                
                y = self.fca2(y)                              
                y = F.relu(y)
                
                y = self.fca3(y)                              
                y = F.relu(y)
                
                net = F.relu(x+y)
                actions_value = self.out(net)                # 给出V(s,a)
                return actions_value


        self.Actor_eval = ANet(s_dim, a_dim, a_bound)
        self.Critic_eval = CNet(s_dim, a_dim)        # 主网络
        self.ctrain = torch.optim.Adam(self.Critic_eval.parameters(),lr = self.lr_c)  # actor的优化器
        self.loss = nn.MSELoss()                          # 损失函数采用均方误差
        
    def train(self):
            
        for i in range(len(self.memory)):
                
            bt = self.memory[i, :]                                              # 采样batch_size个sample
            bs = torch.FloatTensor(bt[:self.s_dim])                                # state
            ba = torch.FloatTensor(bt[self.s_dim: self.s_dim + self.a_dim])        # action
            br = torch.FloatTensor(bt[-self.s_dim - 1: -self.s_dim])
            bs_ = torch.FloatTensor(bt[-self.s_dim:])
#             tmp = np.zeros(q, dtype=np.float32)
            
            a = self.Actor_eval(bs) 
            q = self.Critic_eval(bs, a)
            
            a_ = self.Actor_eval(bs_)
            q_ = self.Critic_eval(bs_, a_)
            
            br += 10
            br = br * 200
            br += 200
            
            loss_td = q - (br + q_)
            
#             loss_td = self.loss(q, -q)
            
            self.ctrain.zero_grad()
            loss_td.backward()
            self.ctrain.step()
        
    def save(self,folder_name):
        os.mkdir('./DPG model/' + folder_name)
        PATH1 = './DPG model/' + folder_name + '/Critic_eval.h5f'
        torch.save(self.Critic_eval.state_dict(), PATH1)
            

In [2]:
###############################  training  ####################################
env = gym.make(ENV_NAME)
env = env.unwrapped
env.seed(SEED)                                          # 设置Gym的随机数种子
torch.manual_seed(SEED)                                 # 设置pytorch的随机数种子

env.set_env_mode(obs_mode='state', reward_type='dense')

s_dim = env.observation_space.shape[0]                  # 状态空间
a_dim = env.action_space.shape[0]                       # 动作空间
a_bound = 1                         # 动作取值区间,对称区间，故只取上界

pre = PreTrain(a_dim, s_dim, a_bound)

pre.Actor_eval.load_state_dict(torch.load('./DPG model/actor_pre/Actor_eval.h5f'))
pre.Actor_eval.eval()

t1 = time.time()

pre.train()

pre.save('critic_pre_r')
print('Running time: ', time.time() - t1)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/lue/anaconda3/envs/mani_skill/lib/python3.8/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Running time:  2418.9997477531433
